In [9]:
import tensorflow as tf
from tasks.model import FacialLandmarkDetector

In [10]:
model = FacialLandmarkDetector()
checkpoint_path = "data/checkpoint_model/cp-{epoch:04d}.ckpt"
epoch = 50
model.load_weights(checkpoint_path.format(epoch=epoch))

Model: "HPE"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(16, 256, 256, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (16, 128, 128, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (16, 128, 128, 64)   256         conv2d[0][0]                     
__________________________________________________________________________________________________
tf.nn.relu (TFOpLambda)         (16, 128, 128, 64)   0           batch_normalization[0][0]        
________________________________________________________________________________________________

In [14]:
from libs import dp
import numpy as np

In [2]:
eval_300W = dp.Dataset(256, 64, 68, '../Data/300W/eval.csv')

Train dataset: ../Data/300W/eval.csv
Train dataset is loaded. Shape: (600, 2)


In [3]:
generator = eval_300W.gen_eval()

In [6]:
for a in generator:
    print(a[0].shape, a[1].shape, a[2], a[3])
    break

(256, 256, 3) (68, 2) [1267.7804999999998, 573.0450000000001] 1.0611367187499998


In [15]:
for image, landmarks, center, scale in generator:
    image = np.array([image])
    heatmaps = model.predict(image)
    break

In [17]:
print(heatmaps.shape)

(1, 64, 64, 68)


In [18]:
from libs.eval import parse_heatmaps

In [20]:
landmarks = parse_heatmaps(heatmaps)

In [22]:
print(landmarks)

[(23, 19), (57, 19), (38, 60), (38, 60), (3, 62), (39, 63), (10, 19), (8, 19), (12, 20), (12, 20), (12, 20), (12, 20), (37, 26), (37, 26), (27, 40), (27, 40), (44, 44), (48, 45), (48, 45), (48, 45), (1, 50), (1, 50), (1, 50), (21, 18), (17, 17), (17, 17), (10, 15), (10, 15), (60, 23), (60, 23), (5, 25), (5, 25), (4, 25), (4, 25), (58, 22), (63, 23), (63, 23), (12, 15), (43, 37), (17, 16), (44, 37), (30, 43), (0, 27), (0, 27), (2, 24), (32, 43), (7, 24), (7, 24), (60, 21), (6, 24), (1, 23), (1, 23), (6, 32), (6, 32), (6, 32), (22, 16), (57, 22), (38, 43), (63, 50), (42, 44), (29, 4), (29, 4), (29, 4), (23, 1)]
